In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 21.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import tensorflow as tf
import keras
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM, BertForNextSentencePrediction, pipeline, BertForQuestionAnswering, BertForTokenClassification, BertForSequenceClassification
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

In [ ]:
#email = pd.read_csv('../../../../../../data/r4.2/email.csv')
#file = pd.read_csv('/content/drive/MyDrive/Praxis/data/file.csv')
#http = pd.read_csv('../../../../../../data/r4.2/http.csv')
#logon = pd.read_csv('../../../../../../data/r4.2/logon.csv')
#device = pd.read_csv('../../../../../../data/r4.2/device.csv')
#tweets = pd.read_csv('/content/drive/MyDrive/Praxis/code/tweets.csv')
#enron = pd.read_csv('../../../../../../data/praxis/ENRON/emails_enron.csv')


In [ ]:
#files = [file,email,http,logon,device]

In [ ]:
#for x in files:
#    print(x.shape)
#file.shape

(445581, 6)

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 15.3 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
from datasets import load_metric, Dataset

In [ ]:
snips_file = pd.read_csv('/content/drive/MyDrive/Praxis/data/train.csv')

In [ ]:
snips_file

,text,slot,intent
0,listen to westbam alumb allergic on google music,O O B-artist O B-album O B-service I-service,PlayMusic
1,add step to me to the 50 clásicos playlist,O B-entity_name I-entity_name I-entity_name O ...,AddToPlaylist
2,i give this current textbook a rating value of...,O O O B-object_select B-object_type O O O O B-...,RateBook
3,play the song little robin redbreast,O O B-music_item B-track I-track I-track,PlayMusic
4,please add iris dement to my playlist this is ...,O O B-artist I-artist O B-playlist_owner O B-p...,AddToPlaylist
...,...,...,...
13079,i want to eat choucroute at a brasserie for 8,O O O O B-served_dish O O B-restaurant_type O ...,BookRestaurant
13080,play funky heavy bluesy,O B-playlist I-playlist I-playlist,PlayMusic
13081,rate the current album 2 points out of 6,O O B-object_select B-object_type B-rating_val...,RateBook
13082,go to the photograph the inflated tear,O O O B-object_type B-object_name I-object_nam...,SearchCreativeWork


In [ ]:
snips_file['intent'].size

13084

In [ ]:
utterances = list(snips_file['text'])

In [ ]:
tokenized_utterances = []
for j in utterances:
  k = j.split(" ")
  tokenized_utterances.append(k)

In [ ]:
tokenized_utterances[0]

['listen', 'to', 'westbam', 'alumb', 'allergic', 'on', 'google', 'music']

In [ ]:
labels_for_tokens = []
for j in list(snips_file['slot']) :
  k = j.split(" ")
  labels_for_tokens.append(k)

In [ ]:
labels_for_tokens[3]

['O', 'O', 'B-music_item', 'B-track', 'I-track', 'I-track']

In [ ]:
print(utterances[0])

listen to westbam alumb allergic on google music


In [ ]:
sequence_labels = list(snips_file['intent'])

In [ ]:
sequence_labels[0]

'PlayMusic'

In [ ]:
len(sequence_labels), len(utterances), len(tokenized_utterances), len(labels_for_tokens)

(13084, 13084, 13084, 13084)

In [ ]:
unique_sequence_labels = list(set(sequence_labels))

In [ ]:
unique_sequence_labels

['PlayMusic',
 'BookRestaurant',
 'SearchScreeningEvent',
 'RateBook',
 'AddToPlaylist',
 'GetWeather',
 'SearchCreativeWork']

In [ ]:
sequence_labels = [unique_sequence_labels.index(l) for l in sequence_labels]

In [ ]:
from functools import reduce

In [ ]:
unique_token_labels = list(set(reduce(lambda x, y: x + y, labels_for_tokens)))

In [ ]:
labels_for_tokens = [[unique_token_labels.index(l) for l in j] for j in labels_for_tokens]

In [ ]:
#unique_token_labels

In [ ]:
from datasets import Dataset

In [ ]:
snips_dataset = Dataset.from_dict(
    dict(
         utterance = utterances,
         label=sequence_labels,
         tokens = tokenized_utterances,
         token_labels = labels_for_tokens

    )
)
snips_dataset = snips_dataset.train_test_split(test_size=.2)

In [ ]:
snips_dataset['train'][0]

{'utterance': 'add xenomania to operación bikini please',
 'label': 4,
 'tokens': ['add', 'xenomania', 'to', 'operación', 'bikini', 'please'],
 'token_labels': [43, 14, 43, 13, 31, 43]}

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
def preprocess_function(examples):
  return tokenizer(examples["utterance"], truncation=True)

In [ ]:
seq_clf_tokenized_snips = snips_dataset.map(preprocess_function, batched=True)

  0%|          | 0/11 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [ ]:
seq_clf_tokenized_snips['train'][0]

{'utterance': 'i give the lacuna a rating of 5 stars under the chronicle rated at 6',
 'label': 3,
 'tokens': ['i',
  'give',
  'the',
  'lacuna',
  'a',
  'rating',
  'of',
  '5',
  'stars',
  'under',
  'the',
  'chronicle',
  'rated',
  'at',
  '6'],
 'token_labels': [43, 43, 19, 57, 43, 43, 43, 24, 18, 43, 43, 66, 43, 43, 38],
 'input_ids': [101,
  1045,
  2507,
  1996,
  18749,
  9521,
  1037,
  5790,
  1997,
  1019,
  3340,
  2104,
  1996,
  9519,
  6758,
  2012,
  1020,
  102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
sequence_clf_model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(unique_sequence_labels))

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'pre_classifi

In [ ]:
sequence_clf_model.config.id2label = {i: l for i, l in enumerate(unique_sequence_labels)}

In [ ]:
sequence_clf_model.config.id2label

{0: 'PlayMusic',
 1: 'BookRestaurant',
 2: 'SearchScreeningEvent',
 3: 'RateBook',
 4: 'AddToPlaylist',
 5: 'GetWeather',
 6: 'SearchCreativeWork'}

In [ ]:
metric = load_metric('accuracy')

<ipython-input-107-343a7ea47872>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('accuracy')


In [ ]:
def compute_metrics(eval_pred):
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=labels)



In [ ]:
epochs = 2

In [ ]:
training_args = TrainingArguments(
    output_dir = "./snips_clf/results",
    num_train_epochs = epochs,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    load_best_model_at_end=True,

    warmup_steps = len(seq_clf_tokenized_snips['train']) // 5,
    weight_decay = .05,

    logging_steps=1,
    log_level='info',
    evaluation_strategy='epoch',
    save_strategy='epoch'
)

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
trainer = Trainer(
    model = sequence_clf_model,
    args = training_args,
    train_dataset=seq_clf_tokenized_snips['train'],
    eval_dataset=seq_clf_tokenized_snips['test'],
    #compute_metrics=compute_metrics,
    data_collator=data_collator
)

In [ ]:
trainer.evaluate()


***** Running Evaluation *****
  Num examples = 2617
  Batch size = 32
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: utterance, token_labels, tokens. If utterance, token_labels, tokens are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.


{'eval_loss': 0.21709804236888885,
 'eval_runtime': 91.6244,
 'eval_samples_per_second': 28.562,
 'eval_steps_per_second': 0.895}

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 10467
  Num Epochs = 2
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 656
  Number of trainable parameters = 66958855
The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: utterance, token_labels, tokens. If utterance, token_labels, tokens are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.


Epoch,Training Loss,Validation Loss
1,0.090100,0.083467
2,0.044100,0.055381


***** Running Evaluation *****
  Num examples = 2617
  Batch size = 32
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: utterance, token_labels, tokens. If utterance, token_labels, tokens are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.


Saving model checkpoint to ./snips_clf/results/checkpoint-328
Configuration saved in ./snips_clf/results/checkpoint-328/config.json
Model weights saved in ./snips_clf/results/checkpoint-328/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2617
  Batch size = 32
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: utterance, token_labels, tokens. If utterance, token_labels, tokens are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
Saving model checkpoint to ./snips_clf/results/checkpoint-656
Configuration saved in ./snips_clf/results/checkpoint-656/config.json
Model weights saved in ./snips_clf/results/checkpoint-656/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./snips_clf/results/checkpoint-656 (score: 0.055381450802087784).


TrainOutput(global_step=656, training_loss=0.1175841016328621, metrics={'train_runtime': 2551.2279, 'train_samples_per_second': 8.205, 'train_steps_per_second': 0.257, 'total_flos': 117088075968786.0, 'train_loss': 0.1175841016328621, 'epoch': 2.0})